In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS

%matplotlib inline
plt.style.use('fivethirtyeight')

C:\Users\nfa\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#Reading in data
index_df = pd.read_excel( r'D:\EME Data\Clean Data\Indices\SP500 FTSE100 GBP.xlsx', date_parser=True)
terror_df = pd.read_excel(r'D:\EME Data\Clean Data\Terror\Uk_Terror_Clean2.xlsx', date_parser=True)

In [3]:
#Merging the dataframes into one
data = index_df.merge(right = terror_df[['Date', 'nkill', 'nwound', 'propvalue', 'incident']] , on = 'Date', how = 'left')
data = data.fillna(0)

In [4]:
print(data['Date'].shape)
print(len(set(data['Date'])))


(8846,)
8846


#Creating a constant
data['const'] = 1

#Creating log variables
data['logFTSE'] = np.log(data['FTSE 100 - PRICE INDEX'])
data['logS&P'] = np.log(data['S&P 500 COMPOSITE - PRICE INDEX (~£ )'])

#Now lagged variables
data['L.logFTSE'] = data['logFTSE'].shift(1)
data['L.logS&P'] = data['logS&P'].shift(1)
data['L.propvalue'] = data['propvalue'].shift(1)

#Creating difference variables
data['D_SP'] = data['logS&P'] - data['L.logS&P']
data['D_FTSE'] = data['logFTSE'] - data['L.logFTSE']

#Creating a dummy variable for if an attack has occured. I do this by dividing the number of incidents a day by itself -
#giving 1 for any incident number not equal to 0 and setting the divide by zero produced NAs to 0 (there's probably a better 
#method that this)
data['terrorattack'] = data['incident']/data['incident']
data['terrorattack'] = data['terrorattack'].fillna(0, axis=0)


#data.to_excel(r'E:\EME Data\Clean Data\Analysis Output\FTSE and Terror merged.xlsx')
##Running ADF test for unit root##

#First we need to slightly change the dataset to the correct format for sm.OLS
regression_df = data.dropna()

#First ADF on non differenced data
ADF_FTSE1 = adfuller(regression_df['logFTSE'], regression= 'nc')
print('ADF_FTSE1 p-value:', ADF_FTSE1[1])
##Therefore we fail to reject the null hypothesis that there is a unit root
ADF_FTSE2 = adfuller(regression_df['D_FTSE'], regression='nc')
print('ADF_FTSE2 p-value:', ADF_FTSE2[1])
##We reject the null hypothesis that differenced log ftse has a unit root. We can conclude that it is an
# I(1) process


##ADF on S&P

ADF_SP1 = adfuller(regression_df['logS&P'], regression= 'nc')
print('ADF_SP1 p-value:', ADF_SP1[1])

ADF_SP2 = adfuller(regression_df['D_SP'], regression='nc')
print('ADF_SP2 p-value:', ADF_SP2[1])
##We find similar conclusions on the S&P 500


##Now for OLS##

#Creating our independent variables matrix
regressors =( regression_df['const'],
              regression_df['D_SP'],
              regression_df['propvalue'],
              regression_df['nwound'],
              regression_df['nkill'] )
X_matrix = np.matrix( (regressors )).T

#Creating a list of regressor names for our output summary
regressor_name = []
for i in np.arange(len(regressors)):
    name = regressors[i].name
    regressor_name = regressor_name + [name]


#Regression1 - All terror variables, no lags
reg1 = OLS(endog=regression_df['D_FTSE'],
          exog = X_matrix).fit(cov_type = 'HC1')
print('Regression 1', reg1.summary(xname=regressor_name))

#Regression2 - All terror variables, two lags each
regressors =( 
              regression_df['D_SP'],
              regression_df['propvalue'],
              regression_df['propvalue'].shift(1),
            regression_df['propvalue'].shift(2),
              regression_df['nwound'],
            regression_df['nwound'].shift(1),
            regression_df['nwound'].shift(2),
              regression_df['nkill'] ,
            regression_df['nkill'].shift(1),
            regression_df['nkill'].shift(2))
X_matrix = np.matrix( (regressors )).T
regressor_name = []
for i in np.arange(len(regressors)):
    name = regressors[i].name
    regressor_name = regressor_name + [name]

reg2 = OLS(endog = regression_df['D_FTSE'],
          exog =X_matrix, missing= 'drop' ).fit(cov_type = 'HC1')
print('Regression 2', reg2.summary(xname=regressor_name))

In [6]:
reg2 = OLS(endog=regression_df['D_FTSE'], exog=regression_df['D_SP']).fit(cov_type= 'HC1')
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:                 D_FTSE   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     469.0
Date:                Tue, 26 Dec 2017   Prob (F-statistic):          2.23e-101
Time:                        14:18:30   Log-Likelihood:                 28379.
No. Observations:                8845   AIC:                        -5.676e+04
Df Residuals:                    8844   BIC:                        -5.675e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
D_SP           0.3652      0.017     21.656      0.0

In [7]:
# print(data.shape, regression_df.shape)
# print(data.columns)
# data.columns = ['Date', 'SP500',
#        'FTSE100', 'nkill', 'nwound', 'propvalue', 'incident',
#        'constant', 'logFTSE', 'logSP', 'LlogFTSE', 'LlogSP', 'Lpropvalue',
#        'D_SP', 'D_FTSE', 'terrorattack']
# data.to_stata(r'E:\EME Data\Clean Data\Python_to_Stata_FTSE_Terror.dta', convert_dates={'Date': 'tc'})